In [3]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# ==========================================
# 1. 데이터 로드
# ==========================================
train_df = pd.read_csv("../Data/train.csv")
test_df = pd.read_csv("../Data/test.csv")

# ==========================================
# 2. 파생변수 생성
# ==========================================

def preprocess(df):
    df_copy = df.copy()
    
    df_copy['배아_생성_효율'] = df_copy['총 생성 배아 수'] / (df_copy['수집된 신선 난자 수'] + 1)
    df_copy['배아_이식_비율'] = df_copy['이식된 배아 수'] / (df_copy['총 생성 배아 수'] + 1)
    df_copy['배아_저장_비율'] = df_copy['저장된 배아 수'] / (df_copy['총 생성 배아 수'] + 1)
    df_copy['나이×Day5'] = (df_copy['시술 당시 나이'].astype(str) + '_' + 
                          (df_copy['배아 이식 경과일'] == 5.0).astype(str)).replace('nan_False', np.nan)

    cat_cols = df_copy.select_dtypes(include=['object', 'category']).columns.tolist()
    for col in cat_cols:
        df_copy[col] = df_copy[col].fillna("None").astype(str)
    
    # 불필요 컬럼 제거 (ID 및 중복 정보)
    drop_cols = ['ID', '이식된 배아 수']
    df_copy = df_copy.drop([c for c in drop_cols if c in df_copy.columns], axis=1)
    
    return df_copy

# ==========================================
# 3. 전처리 적용
# ==========================================
X_train_all = preprocess(train_df)
X_test = preprocess(test_df)

# 타겟 분리
target_col = '임신 성공 여부'
y_train_all = X_train_all[target_col]
X_train_all = X_train_all.drop(columns=[target_col])

# ==========================================
# 4. CatBoost 전용 설정 (Data Leakage 원천 차단)
# ==========================================

# 4-1. 범주형 변수 자동 식별
cat_features = X_train_all.select_dtypes(include=['object', 'category']).columns.tolist()

for col in cat_features:
    X_train_all[col] = X_train_all[col].astype(str).replace('nan', np.nan)
    X_test[col] = X_test[col].astype(str).replace('nan', np.nan)

# 4-2. 학습/검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(
    X_train_all, y_train_all, 
    test_size=0.2, 
    random_state=42, 
    stratify=y_train_all
)

train_pool = Pool(data=X_train, label=y_train, cat_features=cat_features)
val_pool = Pool(data=X_val, label=y_val, cat_features=cat_features)
test_pool = Pool(data=X_test, cat_features=cat_features)

# ==========================================
# 5. 모델 학습
# ==========================================

cat_model = CatBoostClassifier(
    iterations=3000,             # 충분한 학습 횟수
    learning_rate=0.03,          # 낮추어 정밀하게 학습
    depth=6,                     # 과적합 방지를 위해 적절한 깊이
    loss_function='Logloss',     # 이진 분류 손실 함수
    eval_metric='AUC',           # 평가 지표
    random_seed=42,
    task_type="CPU",             
    thread_count=-1,             # 모든 코어 사용
    early_stopping_rounds=200,   # 200회 동안 성능 향상 없으면 중단
    verbose=500                  # 500회마다 로그 출력
)

print("CatBoost 학습 시작...")
cat_model.fit(train_pool, eval_set=val_pool, use_best_model=True)

# ==========================================
# 6. 예측 (Positive Probability)
# ==========================================
print("\n예측 수행 중...")
# 원칙 1,2 준수: Test Data는 오직 예측 단계에서만 사용됨
pred_probs = cat_model.predict_proba(test_pool)

# Positive Class(1)에 대한 확률만 추출
final_probs = pred_probs[:, 1]

# 결과 확인
print(f"학습 종료. Validation Best AUC: {cat_model.get_best_score()['validation']['AUC']:.5f}")
print(f"생성된 확률 값 예시 (상위 5개): {final_probs[:5]}")
print(f"데이터 개수 확인: {len(final_probs)}")

CatBoost 학습 시작...
0:	test: 0.6940509	best: 0.6940509 (0)	total: 123ms	remaining: 6m 9s
500:	test: 0.7361346	best: 0.7361346 (500)	total: 54.1s	remaining: 4m 29s
1000:	test: 0.7366272	best: 0.7366305 (998)	total: 1m 51s	remaining: 3m 41s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.7366999776
bestIteration = 1073

Shrink model to first 1074 iterations.

예측 수행 중...
학습 종료. Validation Best AUC: 0.73670
생성된 확률 값 예시 (상위 5개): [0.00166579 0.00328929 0.1448807  0.1096035  0.51821707]
데이터 개수 확인: 90067


In [4]:
# ==========================================
# 결과 추출
# ==========================================
from datetime import datetime
now = datetime.now().strftime('%m%d_%H%M')
file_name = f"CB_{now}_submission.csv"

submission = pd.read_csv("../Data/sample_submission.csv")
submission['probability'] = final_probs
submission.to_csv(file_name, index=False)

print(f"완료! 결과 저장됨: {file_name}")

완료! 결과 저장됨: CB_0211_0001_submission.csv
